In [1]:
%load_ext autoreload
%autoreload 2
from PyHa.statistics import *
from PyHa.IsoAutio import *
from PyHa.visualizations import *
from PyHa.annotation_post_processing import *
import pandas as pd

In [2]:
path = "./mixed_bird/025_Mixed_Bird_Outputs_Train/"

In [3]:
from PyHa.tweetynet_package.tweetynet.Load_data_functions import compute_features, predictions_to_kaleidoscope
from PyHa.microfaune_package.microfaune import audio
import torch.optim as optim
import torch.nn as nn

# PREP

In [4]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
class kaledoscope_dataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file,filepath, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.manual_df = pd.read_csv(csv_file)
        self.manual_df["FOLDER"] = self.manual_df["IN FILE"].apply(lambda x: filepath)
        self.audio_files = self.manual_df[["FOLDER","IN FILE"]].drop_duplicates().reset_index(drop=True)
        self.transform = transform
    def __len__(self):
        return len(self.manual_df)

    def __getitem__(self, idx):
        #TODO: FIX MAGIC NUMBERS
        normalized_sample_rate=44100

        audio_df = self.audio_files.iloc[idx]
        
        audio_dif = audio_df["FOLDER"]
        audio_file = audio_df["IN FILE"]
        try:
            SAMPLE_RATE, SIGNAL = audio.load_wav(audio_dif + audio_file)
        except BaseException as e:
            #print("Failed to load", audio_dif + audio_file)
            #print(e)
            return (-1,-1)
        
        try:
            if SAMPLE_RATE != normalized_sample_rate:
                rate_ratio = normalized_sample_rate / SAMPLE_RATE
                SIGNAL = scipy_signal.resample(
                    SIGNAL, int(len(SIGNAL) * rate_ratio))
                SAMPLE_RATE = normalized_sample_rate
        except:
            print("Failed to Downsample" + audio_file)
            return (-1,-1)
            
        if len(SIGNAL.shape) == 2:
            SIGNAL = SIGNAL.sum(axis=1) / 2
        # detection
        try:
            tweetynet_features = compute_features([SIGNAL])
        except:
            print("Failed to compute features" + audio_file)
            return (-1,-1)
        

            
        label_df = self.manual_df[(self.manual_df["FOLDER"] == audio_df["FOLDER"]) & (self.manual_df["IN FILE"] == audio_df["IN FILE"])]
        return (tweetynet_features, label_df, audio_dif, audio_file, SIGNAL, SAMPLE_RATE)
    
trainloader = kaledoscope_dataset("mixed_bird_manual.csv", path)
trainloader

In [5]:
# init detector
device = torch.device('cpu')
net_wrapper = TweetyNetModel(2, (1, 86, 86), 86, device)

# Training

In [6]:
# TODO: DEFINE TEST LOADER

In [7]:
def IOU_Loss(automated_df, manual_df):
    IoUMatrix = torch.tensor(clip_IoU(automated_df, manual_df));
    return torch.mean(IoUMatrix)

def convert_label_to_local_score(manual_df, size_of_local_sorce):
    duration_of_clip = manual_df.iloc[0]["CLIP LENGTH"]
    seconds_per_index = duration_of_clip/size_of_local_sorce
    local_score = np.zeros(size_of_local_sorce)
    for i in range(size_of_local_sorce):
        current_seconds = i * seconds_per_index
        annotations_at_time = manual_df[(manual_df["OFFSET"] <= current_seconds) & (manual_df["OFFSET"] +manual_df["DURATION"] >=  current_seconds)]
        if (not annotations_at_time.empty):
            local_score[i] = 1
    
    return torch.tensor(local_score)

manual_df  = pd.read_csv("ScreamingPiha_Manual_Labels.csv")
convert_label_to_local_score(manual_df[manual_df["IN FILE"] == "ScreamingPiha5.wav"], 500)

tensor([0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1.,
        1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0.,
        0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
        0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [11]:
def generate_output(test_dataset):
        #print("Generate output")
        device = torch.device('cpu')
        net = net_wrapper.model
        for param in net.parameters():
            param.requires_grad = True
        
        batch_size=1
        window_size=2
        test_data_loader = DataLoader(test_dataset, batch_size=batch_size)
        predictions = pd.DataFrame()
        net_wrapper.model.eval()
        local_score = torch.tensor([])
        tmp_local_score = []
        dataiter = iter(test_data_loader)
        _, label, uid = dataiter.next()
        time_bin = float(window_size)/label.shape[1]
        st_time = np.array([time_bin*n for n in range(label.shape[1])])
        
        for i, data in enumerate(test_data_loader):
                inputs, labels, uids = data
                inputs = inputs.clone().detach().requires_grad_(True)
                inputs, labels = inputs.to(device), labels.to(device)
                #print(inputs)
                output = net_wrapper.model(inputs, inputs.shape[0], inputs.shape[0])
                #print(output)
                pred_arg = torch.argmax(output, dim=1)
                pred_sof = torch.softmax(output, dim=1)
                #LogSoftMax???
                ##print(pred_sof)
                ##print(pred_sof[0, 1, :])
                #pred = pred_sof.reshape(pred_arg.shape)
                #print(pred.shape[0])
                #print(pred.shape[1])
                #local_score = torch.cat((local_score, pred))
                bins = st_time + (int(uids[0].split("_")[0])*window_size)
                local_score = torch.cat((local_score, pred_sof[0, 1, :]))#output[0, 1, :]))
                tmp_local_score.extend([x for x in pred_sof[0, 1, :]])#output[0, 1, :]])
                #pred = pred.reshape(pred.shape[1])
                #labels = labels.reshape(labels.shape[1])
                #bins = st_time + (int(uids[0].split("_")[0])*window_size)
                #d = {"uid": uids[0], "pred": pred, "label": labels, "time_bins": bins}
                #new_preds = pd.DataFrame(d)
                #predictions = predictions.append(new_preds)
        
        #print("output", output)
        #print("local_scores", local_score)
        #print("local_scores", local_score.size(0))
        
        #print("===========")
        #print(output[0, 1, :])
        local_score = local_score.to(torch.float64)
                
        return local_score, len(tmp_local_score)
    
for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        try:
            inputs, labels, audio_dif, audio_file, SIGNAL, SAMPLE_RATE = data
        except:
            continue
        generate_output(inputs)
        break;


In [12]:
net = net_wrapper.model
print(list(net.parameters())[0].grad)

None


In [14]:
net = net_wrapper.model
for param in net.parameters():
    param.requires_grad = True
optimizer = optim.Adam(net.parameters(), lr=0.01)
criterion = nn.MSELoss()

for epoch in range(1):  # loop over the dataset multiple times
    running_loss = 0.0
    batch_loss = 0.0
    mini_batch_count = 0
    batch_count = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        try:
            inputs, labels, audio_dif, audio_file, SIGNAL, SAMPLE_RATE = data
        except:
            continue

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        output_local_score, size = generate_output(inputs)
        input_local_score = convert_label_to_local_score(labels, size)
        
        print(output_local_score)
        print(input_local_score)
        
        loss = criterion(output_local_score, input_local_score)
        #print(loss, loss.item())
        #print(net_wrapper.model.state_dict())
        state_a = net_wrapper.model.state_dict().__str__()
        #print(list(net.parameters())[0][0])
        loss.backward()
        optimizer.step()
        #print(list(net.parameters())[0][0])
        state_b = net_wrapper.model.state_dict().__str__()
        if state_a == state_b:
            print("Network not updating.")

        # print statistics
        running_loss += loss.item()
        batch_loss += loss.item()
        mini_batch_count += 1
        batch_count += 1
        print(batch_count)
        print(f'[{epoch + 1, mini_batch_count}] loss: {running_loss / mini_batch_count:.3f}')
        if (mini_batch_count == 10):
            print(f'[{epoch + 1, mini_batch_count}] loss: {running_loss / mini_batch_count:.3f}')
            running_loss = 0.0
            mini_batch_count = 0
            break
    print(f'[{epoch + 1, batch_count}] loss: {batch_loss / batch_count:.3f}')
    batch_loss = 0.0
    batch_count = 0

print('Finished Training')
torch.save(net.state_dict(), "./test.h5")

tensor([0.4895, 0.4925, 0.4945, 0.4949, 0.4950, 0.4946, 0.4947, 0.4947, 0.4948,
        0.4949, 0.4948, 0.4948, 0.4948, 0.4949, 0.4951, 0.4951, 0.4951, 0.4949,
        0.4950, 0.4948, 0.4947, 0.4947, 0.4948, 0.4948, 0.4948, 0.4948, 0.4949,
        0.4949, 0.4948, 0.4945, 0.4944, 0.4946, 0.4946, 0.4946, 0.4948, 0.4949,
        0.4949, 0.4948, 0.4946, 0.4944, 0.4943, 0.4945, 0.4946, 0.4948, 0.4949,
        0.4950, 0.4948, 0.4946, 0.4944, 0.4944, 0.4945, 0.4948, 0.4952, 0.4953,
        0.4953, 0.4951, 0.4949, 0.4948, 0.4949, 0.4948, 0.4949, 0.4950, 0.4951,
        0.4949, 0.4947, 0.4945, 0.4944, 0.4944, 0.4947, 0.4949, 0.4951, 0.4950,
        0.4947, 0.4943, 0.4942, 0.4944, 0.4944, 0.4947, 0.4950, 0.4951, 0.4950,
        0.4946, 0.4943, 0.4950, 0.4952, 0.4943, 0.4900, 0.4929, 0.4945, 0.4944,
        0.4947, 0.4946, 0.4946, 0.4946, 0.4947, 0.4949, 0.4952, 0.4952, 0.4950,
        0.4945, 0.4942, 0.4943, 0.4944, 0.4945, 0.4948, 0.4951, 0.4951, 0.4949,
        0.4946, 0.4942, 0.4943, 0.4943, 

2
[(1, 2)] loss: 0.271
tensor([0.5541, 0.5727, 0.5850, 0.5901, 0.5923, 0.5928, 0.5926, 0.5923, 0.5922,
        0.5922, 0.5921, 0.5920, 0.5917, 0.5916, 0.5914, 0.5913, 0.5913, 0.5914,
        0.5915, 0.5919, 0.5920, 0.5920, 0.5919, 0.5918, 0.5916, 0.5916, 0.5914,
        0.5915, 0.5915, 0.5916, 0.5916, 0.5917, 0.5916, 0.5916, 0.5915, 0.5914,
        0.5914, 0.5914, 0.5915, 0.5915, 0.5916, 0.5917, 0.5918, 0.5918, 0.5919,
        0.5918, 0.5917, 0.5916, 0.5914, 0.5915, 0.5914, 0.5913, 0.5913, 0.5914,
        0.5914, 0.5915, 0.5918, 0.5919, 0.5919, 0.5917, 0.5917, 0.5915, 0.5914,
        0.5914, 0.5914, 0.5913, 0.5913, 0.5913, 0.5914, 0.5917, 0.5921, 0.5923,
        0.5923, 0.5922, 0.5920, 0.5916, 0.5915, 0.5913, 0.5912, 0.5912, 0.5910,
        0.5905, 0.5891, 0.5862, 0.5772, 0.5616, 0.5572, 0.5762, 0.5879, 0.5926,
        0.5941, 0.5940, 0.5938, 0.5935, 0.5933, 0.5930, 0.5927, 0.5922, 0.5919,
        0.5917, 0.5915, 0.5915, 0.5916, 0.5917, 0.5920, 0.5922, 0.5924, 0.5924,
        0.5924, 0

5
[(1, 5)] loss: 0.257
tensor([0.3616, 0.3161, 0.2794, 0.2590, 0.2454, 0.2370, 0.2324, 0.2298, 0.2282,
        0.2272, 0.2266, 0.2263, 0.2260, 0.2257, 0.2256, 0.2253, 0.2252, 0.2253,
        0.2253, 0.2254, 0.2256, 0.2256, 0.2254, 0.2253, 0.2251, 0.2248, 0.2244,
        0.2242, 0.2240, 0.2237, 0.2239, 0.2242, 0.2244, 0.2246, 0.2247, 0.2246,
        0.2245, 0.2245, 0.2244, 0.2245, 0.2246, 0.2249, 0.2251, 0.2254, 0.2255,
        0.2259, 0.2261, 0.2262, 0.2263, 0.2265, 0.2266, 0.2263, 0.2262, 0.2261,
        0.2260, 0.2259, 0.2259, 0.2260, 0.2260, 0.2258, 0.2255, 0.2252, 0.2245,
        0.2232, 0.2217, 0.2202, 0.2190, 0.2186, 0.2188, 0.2196, 0.2204, 0.2213,
        0.2226, 0.2237, 0.2248, 0.2256, 0.2266, 0.2280, 0.2295, 0.2315, 0.2348,
        0.2399, 0.2477, 0.2623, 0.2923, 0.3330], dtype=torch.float64,
       grad_fn=<ToCopyBackward0>)
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

11
[(1, 11)] loss: 0.291
tensor([0.3584, 0.3370, 0.3344, 0.3325, 0.3326, 0.3326, 0.3324, 0.3323, 0.3323,
        0.3324, 0.3324, 0.3325, 0.3327, 0.3328, 0.3328, 0.3328, 0.3329, 0.3328,
        0.3328, 0.3328, 0.3327, 0.3325, 0.3325, 0.3325, 0.3325, 0.3325, 0.3325,
        0.3325, 0.3325, 0.3325, 0.3326, 0.3326, 0.3326, 0.3327, 0.3328, 0.3327,
        0.3327, 0.3328, 0.3331, 0.3334, 0.3338, 0.3343, 0.3347, 0.3347, 0.3346,
        0.3346, 0.3344, 0.3340, 0.3338, 0.3337, 0.3335, 0.3336, 0.3338, 0.3340,
        0.3343, 0.3346, 0.3347, 0.3348, 0.3348, 0.3347, 0.3345, 0.3343, 0.3340,
        0.3339, 0.3338, 0.3337, 0.3336, 0.3337, 0.3337, 0.3339, 0.3341, 0.3343,
        0.3344, 0.3344, 0.3345, 0.3343, 0.3341, 0.3338, 0.3334, 0.3331, 0.3327,
        0.3323, 0.3319, 0.3305, 0.3250, 0.3225, 0.3571, 0.3394, 0.3364, 0.3348,
        0.3347, 0.3343, 0.3339, 0.3336, 0.3334, 0.3331, 0.3330, 0.3329, 0.3330,
        0.3331, 0.3334, 0.3338, 0.3340, 0.3342, 0.3343, 0.3343, 0.3341, 0.3339,
        0.3336,

13
[(1, 13)] loss: 0.280
tensor([0.4208, 0.4088, 0.4057, 0.4040, 0.4038, 0.4037, 0.4037, 0.4036, 0.4036,
        0.4036, 0.4036, 0.4036, 0.4036, 0.4036, 0.4036, 0.4036, 0.4036, 0.4036,
        0.4036, 0.4036, 0.4036, 0.4036, 0.4036, 0.4036, 0.4036, 0.4036, 0.4036,
        0.4036, 0.4036, 0.4036, 0.4036, 0.4036, 0.4036, 0.4036, 0.4036, 0.4036,
        0.4036, 0.4036, 0.4036, 0.4036, 0.4036, 0.4036, 0.4036, 0.4036, 0.4036,
        0.4036, 0.4036, 0.4036, 0.4036, 0.4036, 0.4036, 0.4036, 0.4036, 0.4036,
        0.4036, 0.4036, 0.4036, 0.4036, 0.4036, 0.4035, 0.4036, 0.4035, 0.4036,
        0.4035, 0.4036, 0.4035, 0.4035, 0.4035, 0.4035, 0.4035, 0.4035, 0.4035,
        0.4035, 0.4035, 0.4035, 0.4035, 0.4035, 0.4034, 0.4034, 0.4032, 0.4031,
        0.4029, 0.4027, 0.4019, 0.3974, 0.3865, 0.4207, 0.4089, 0.4054, 0.4040,
        0.4038, 0.4036, 0.4036, 0.4036, 0.4036, 0.4036, 0.4035, 0.4036, 0.4036,
        0.4036, 0.4036, 0.4036, 0.4035, 0.4035, 0.4035, 0.4035, 0.4035, 0.4035,
        0.4035,

...

79
[(1, 79)] loss: 0.247
tensor([0.3607, 0.3455, 0.3432, 0.3429, 0.3428, 0.3428, 0.3429, 0.3429, 0.3429,
        0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429,
        0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429,
        0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429,
        0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429,
        0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429,
        0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429,
        0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429,
        0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429,
        0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3597, 0.3454, 0.3432, 0.3429,
        0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429,
        0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429, 0.3429,
        0.3429,

82
[(1, 82)] loss: 0.248
tensor([0.3606, 0.3463, 0.3441,  ..., 0.3438, 0.3438, 0.3437],
       dtype=torch.float64, grad_fn=<ToCopyBackward0>)
tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64)
83
[(1, 83)] loss: 0.248
tensor([0.3612, 0.3468, 0.3446,  ..., 0.3443, 0.3443, 0.3442],
       dtype=torch.float64, grad_fn=<ToCopyBackward0>)
tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64)
84
[(1, 84)] loss: 0.247
tensor([0.3615, 0.3473, 0.3452,  ..., 0.3448, 0.3448, 0.3448],
       dtype=torch.float64, grad_fn=<ToCopyBackward0>)
tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64)
85
[(1, 85)] loss: 0.247
tensor([0.3619, 0.3478, 0.3457,  ..., 0.3453, 0.3453, 0.3453],
       dtype=torch.float64, grad_fn=<ToCopyBackward0>)
tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64)
86
[(1, 86)] loss: 0.246
tensor([0.3620, 0.3477, 0.3455, 0.3452, 0.3452, 0.3452, 0.3452, 0.3452, 0.3452,
        0.3452, 0.3452, 0.3452, 0.3452, 0.3452, 0.3452, 0.3452, 0.3452, 0.3452,
   

93
[(1, 93)] loss: 0.243
tensor([0.3531, 0.3382, 0.3360,  ..., 0.3356, 0.3356, 0.3355],
       dtype=torch.float64, grad_fn=<ToCopyBackward0>)
tensor([0., 0., 1.,  ..., 0., 0., 0.], dtype=torch.float64)
94
[(1, 94)] loss: 0.244
tensor([0.3535, 0.3385, 0.3362, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359,
        0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359,
        0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359,
        0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359,
        0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359,
        0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359,
        0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359,
        0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359,
        0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359, 0.3359,
        0.3359, 0.3359, 0.3359, 0.3358, 0.3358, 0.35

96
[(1, 96)] loss: 0.243
tensor([0.3532, 0.3381, 0.3359, 0.3356, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355,
        0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355,
        0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355,
        0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355,
        0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355,
        0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355,
        0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355,
        0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355,
        0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355,
        0.3355, 0.3355, 0.3355, 0.3355, 0.3354, 0.3530, 0.3381, 0.3359, 0.3356,
        0.3356, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355,
        0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355, 0.3355,
        0.3355,

99
[(1, 99)] loss: 0.245
tensor([0.3587, 0.3441, 0.3419, 0.3417, 0.3416, 0.3416, 0.3416, 0.3416, 0.3416,
        0.3416, 0.3416, 0.3416, 0.3416, 0.3416, 0.3417, 0.3417, 0.3417, 0.3417,
        0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417,
        0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417,
        0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417,
        0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417,
        0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417,
        0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417,
        0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417,
        0.3417, 0.3417, 0.3417, 0.3416, 0.3416, 0.3585, 0.3442, 0.3420, 0.3417,
        0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417,
        0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417, 0.3417,
        0.3417,

In [ ]:
%reload_ext autoreload
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs


isolation_parameters = {
     "model" : "tweetynet",
     "tweety_output": True,
    "technique" : "steinberg",
     "threshold_type" : "median",
     "threshold_const" : 2.0,
     "threshold_min" : 0.0,
     "window_size" : 2.0,
     "chunk_size" : 5.0
}
automated_df = generate_automated_labels(path,isolation_parameters, weight_path="./test.h5");
display(automated_df)
manual_df = pd.read_csv("mixed_bird_manual.csv")
statistics_df = automated_labeling_statistics(automated_df,manual_df,stats_type = "general");
statistics_df
global_dataset_statistics(statistics_df)